# PILS Examples Notebook

Comprehensive examples for using PILS (POLOCALC Inertial & Drone Loading System).

## Table of Contents

1. [Loading Flight Data](#1-loading-flight-data)
2. [Working with Sensors](#2-working-with-sensors)
3. [Drone Data](#3-drone-data)
4. [Time Synchronization](#4-time-synchronization)
5. [PPK Analysis](#5-ppk-analysis)
6. [Data Export](#6-data-export)
7. [HDF5 Operations](#7-hdf5-operations)

In [ ]:
# Import required libraries
import polars as pl
from pathlib import Path
from pils.flight import Flight
from pils.loader import PathLoader
from pils.synchronizer import CorrelationSynchronizer
from pils.analyze.ppk import PPKAnalyzer

print("PILS imported successfully!")

## 1. Loading Flight Data

### Using PathLoader for Local Files

In [ ]:
# Initialize loader
loader = PathLoader()

# Load a single flight
# Replace with your actual flight path
flight_path = Path("/path/to/flight/data")

# Uncomment to load:
# flight = loader.load_single_flight(data_path=flight_path)
# print(f"Flight ID: {flight.flight_info['flight_id']}")

# Or create a flight manually
flight = Flight(flight_info={
    'flight_id': 'example-001',
    'date': '2024-01-15',
    'pilot': 'John Doe',
    'location': 'Test Site'
})

print("Flight created:", flight.flight_info)

## 2. Working with Sensors

### Loading GPS Data

In [ ]:
from pils.sensors.gps import GPS

# Create sample GPS data
sample_gps = pl.DataFrame({
    'timestamp': [1700000000000000, 1700000001000000, 1700000002000000],
    'lat': [40.7128, 40.7129, 40.7130],
    'lon': [-74.0060, -74.0061, -74.0062],
    'altitude': [10.5, 10.6, 10.7],
    'speed': [0.0, 0.5, 1.0],
    'fix_type': [4, 4, 4],
    'satellite_count': [12, 12, 12]
})

print("GPS Data Schema:")
print(sample_gps)

# Filter by fix quality
fixed = sample_gps.filter(pl.col('fix_type') >= 4)
print(f"\nFixed solutions: {fixed.height}/{sample_gps.height}")

### Loading IMU Data

In [ ]:
# Create sample IMU data
sample_imu = pl.DataFrame({
    'timestamp': [1700000000000000, 1700000000010000, 1700000000020000],
    'acc_x': [0.01, 0.02, 0.01],
    'acc_y': [-0.02, -0.01, -0.02],
    'acc_z': [1.01, 1.00, 1.01],
    'gyro_x': [0.5, 0.4, 0.5],
    'gyro_y': [-0.3, -0.2, -0.3],
    'gyro_z': [0.1, 0.2, 0.1],
    'temperature': [25.5, 25.5, 25.6]
})

print("IMU Data:")
print(sample_imu)

# Calculate acceleration magnitude
sample_imu = sample_imu.with_columns([
    (pl.col('acc_x')**2 + pl.col('acc_y')**2 + pl.col('acc_z')**2)
    .sqrt()
    .alias('acc_mag')
])

print("\nWith acceleration magnitude:")
print(sample_imu.select(['timestamp', 'acc_mag']))

## 3. Drone Data

### Working with DJI Telemetry

In [ ]:
# Create sample DJI data
sample_dji = pl.DataFrame({
    'frame_number': [1, 2, 3],
    'lat': [40.7128, 40.7129, 40.7130],
    'lon': [-74.0060, -74.0061, -74.0062],
    'rel_alt': [10.0, 20.0, 30.0],
    'h_speed': [0.0, 2.5, 5.0],
    'v_speed': [0.0, 1.0, 1.5],
    'iso': [100, 100, 200],
    'gimbal_pitch': [-45.0, -45.0, -45.0]
})

print("DJI Telemetry:")
print(sample_dji)

# Convert speed to km/h
sample_dji = sample_dji.with_columns([
    (pl.col('h_speed') * 3.6).alias('h_speed_kmh'),
    (pl.col('v_speed') * 3.6).alias('v_speed_kmh')
])

print("\nWith speeds in km/h:")
print(sample_dji.select(['frame_number', 'h_speed_kmh', 'v_speed_kmh']))

## 4. Time Synchronization

### Using CorrelationSynchronizer

In [ ]:
# Simulate sensor data with time offsets
base_time = 1700000000000000

# GPS data (reference)
gps_data = pl.DataFrame({
    'timestamp': [base_time + i*1000000 for i in range(10)],
    'lat': [40.7128 + i*0.0001 for i in range(10)],
    'lon': [-74.0060 + i*0.0001 for i in range(10)]
})

# IMU data (with 500ms offset)
imu_data = pl.DataFrame({
    'timestamp': [base_time + 500000 + i*100000 for i in range(100)],
    'acc_x': [0.01] * 100,
    'acc_z': [1.0] * 100
})

print(f"GPS samples: {gps_data.height}")
print(f"IMU samples: {imu_data.height}")
print(f"GPS time range: {gps_data['timestamp'].min()} - {gps_data['timestamp'].max()}")
print(f"IMU time range: {imu_data['timestamp'].min()} - {imu_data['timestamp'].max()}")

# Note: Actual synchronization requires complete Flight object
# sync = CorrelationSynchronizer()
# flight = sync.synchronize(flight)

## 5. PPK Analysis

### Post-Processed Kinematic Positions

In [ ]:
from datetime import datetime

# Create sample PPK position data
sample_ppk = pl.DataFrame({
    'datetime': [
        datetime(2024, 1, 15, 12, 0, 0),
        datetime(2024, 1, 15, 12, 0, 1),
        datetime(2024, 1, 15, 12, 0, 2)
    ],
    'lat': [40.712800, 40.712801, 40.712802],
    'lon': [-74.006000, -74.006001, -74.006002],
    'height': [10.500, 10.510, 10.520],
    'Q': [1, 1, 1],  # Fixed solution
    'ns': [12, 12, 12],  # Number of satellites
    'sdn': [0.010, 0.010, 0.010],  # North std dev (m)
    'sde': [0.008, 0.008, 0.008],  # East std dev (m)
    'sdu': [0.020, 0.020, 0.020],  # Up std dev (m)
    'ratio': [5.2, 5.3, 5.1]  # Ambiguity ratio
})

print("PPK Position Data:")
print(sample_ppk)

# Quality analysis
quality_summary = sample_ppk.groupby('Q').agg([
    pl.count().alias('count'),
    pl.col('sdn').mean().alias('mean_sdn'),
    pl.col('sde').mean().alias('mean_sde')
])

print("\nQuality Summary:")
print(quality_summary)

# Calculate horizontal accuracy
sample_ppk = sample_ppk.with_columns([
    (pl.col('sdn')**2 + pl.col('sde')**2).sqrt().alias('h_accuracy')
])

print("\nHorizontal Accuracy (m):")
print(sample_ppk.select(['datetime', 'Q', 'h_accuracy']))

## 6. Data Export

### Exporting to Various Formats

In [ ]:
# Using the sample GPS data from earlier
export_data = sample_gps

# Export to CSV
# export_data.write_csv("output/gps_export.csv")
print("CSV export format:")
print(export_data.write_csv()[:200])  # Show first 200 chars

# Export to Parquet (efficient binary format)
# export_data.write_parquet("output/gps_export.parquet")
print("\nParquet format: Efficient binary storage with compression")

# Export to JSON
# export_data.write_json("output/gps_export.json")
print("\nJSON export format (first record):")
print(export_data.head(1).write_json())

## 7. HDF5 Operations

### Save and Load Flight Data

In [ ]:
# Create a complete flight object
demo_flight = Flight(flight_info={
    'flight_id': 'demo-flight-001',
    'date': '2024-01-15',
    'pilot': 'Demo Pilot',
    'location': 'Example Site'
})

# Add sample sensor data
from pils.sensors.gps import GPS

# Note: In real usage, load from files:
# demo_flight.add_sensor_data(['gps', 'imu'])

print("Flight info:")
print(demo_flight.flight_info)

# Save to HDF5 (uncomment to actually save)
# demo_flight.to_hdf5('output/demo_flight.h5')
# print("\nFlight saved to HDF5")

# Load from HDF5
# loaded_flight = Flight.from_hdf5('output/demo_flight.h5')
# print("\nFlight loaded from HDF5:")
# print(loaded_flight.flight_info)

## Advanced: Data Analysis Example

### GPS Trajectory Analysis

In [ ]:
# Create a longer GPS trajectory
import numpy as np

n_points = 100
trajectory = pl.DataFrame({
    'timestamp': [1700000000000000 + i*1000000 for i in range(n_points)],
    'lat': 40.7128 + np.cumsum(np.random.randn(n_points) * 0.00001),
    'lon': -74.0060 + np.cumsum(np.random.randn(n_points) * 0.00001),
    'altitude': 10.0 + np.cumsum(np.random.randn(n_points) * 0.1)
})

print(f"Trajectory with {n_points} points")
print("\nFirst 5 points:")
print(trajectory.head())

# Calculate distances between points (simplified)
trajectory = trajectory.with_columns([
    pl.col('lat').diff().alias('dlat'),
    pl.col('lon').diff().alias('dlon')
])

# Calculate total distance (approximate)
trajectory = trajectory.with_columns([
    ((pl.col('dlat')**2 + pl.col('dlon')**2).sqrt() * 111320).alias('distance_m')
])

total_distance = trajectory['distance_m'].sum()
print(f"\nApproximate total distance: {total_distance:.1f} meters")

# Altitude statistics
print("\nAltitude statistics:")
print(f"Min: {trajectory['altitude'].min():.2f} m")
print(f"Max: {trajectory['altitude'].max():.2f} m")
print(f"Mean: {trajectory['altitude'].mean():.2f} m")
print(f"Std: {trajectory['altitude'].std():.2f} m")

## Summary

This notebook demonstrated:

1. ✅ Loading flight data with PathLoader
2. ✅ Working with GPS and IMU sensors
3. ✅ Processing drone telemetry
4. ✅ Time synchronization concepts
5. ✅ PPK position analysis
6. ✅ Data export to multiple formats
7. ✅ HDF5 save/load operations
8. ✅ Advanced trajectory analysis

### Next Steps

- Check the [documentation](https://polocalc.github.io/pils/) for more details
- See other example files in this folder
- Try with your own flight data

### Resources

- [PILS Documentation](https://polocalc.github.io/pils/)
- [Polars Documentation](https://pola-rs.github.io/polars/)
- [GitHub Repository](https://github.com/polocalc/pils)